### Milling Tool Environment R&D and SB-3 algoithm test
-----------------------------------------------------------------------------

- V.2: 28-Apr-2023 
    * Adhere to Open AI gym template and test with SB-3 PPO
    * 'info' argument - change to dictionary 

1. Improvements to Milling Tool env.
2. Test using Stable Baseline algorithms
3. On finalizing - move to .py module and import in custom algorithms

In [8]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [21]:
import gym
import pandas as pd
import numpy as np
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C, PPO, DQN

import milling_tool_environment
from milling_tool_environment import MillingTool

In [12]:
from stable_baselines3.common.env_checker import check_env
WEAR_THRESHOLD = 3.0
WEAR_MAX_ALLOWED = 1.3 # 30%
TOOL_REPLACEMENT_IDEAL = 70

df = pd.read_csv('Tool_Wear_VB.csv')
df = df[['time', 'VB_mm', 'ACTION_CODE']]
print('Tool wear data imported. Number of records: ', len(df.index),'\n')

env = MillingTool(df, WEAR_THRESHOLD, WEAR_MAX_ALLOWED, TOOL_REPLACEMENT_IDEAL)
check_env(env)

env.close()

Tool wear data imported. Number of records:  121 



In [13]:
#EPISODES = 400
WEAR_THRESHOLD = 3.0
WEAR_MAX_ALLOWED = 1.3 # 30%
TOOL_REPLACEMENT_IDEAL = 70

df = pd.read_csv('Tool_Wear_VB.csv')
df = df[['time', 'VB_mm', 'ACTION_CODE']]
print('Tool wear data imported. Number of records: ', len(df.index),'\n')

df = (df-df.min())/(df.max()-df.min())
print(df.head())

Tool wear data imported. Number of records:  121 

       time     VB_mm  ACTION_CODE
0  0.000000  0.000000          0.0
1  0.008333  0.004772          0.0
2  0.016667  0.006209          0.0
3  0.025000  0.007337          0.0
4  0.033333  0.008336          0.0


In [14]:
# The algorithms require a vectorized environment to run
env = MillingTool(df, WEAR_THRESHOLD, WEAR_MAX_ALLOWED, TOOL_REPLACEMENT_IDEAL)

model = PPO('MlpPolicy', env, verbose=1)

model.learn(total_timesteps=300)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 1585 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------


In [17]:
test_cases = np.random.randint(0, env.df_length-1, 100)
cumm_error = 0.0
for idx in test_cases:
    state = env._get_observation(idx)
    action_pred, next_state = model.predict(state)
    action_actual = int(df['ACTION_CODE'][idx])
    e = int(action_pred - action_actual)
    cumm_error += abs(e)
    #print(f'{idx:4d}: Action predicted: {action_pred} | actual: {action_actual} | error: {e}')
    
print(f'\n Avg. Error over {len(test_cases)} test cases: {cumm_error*100.00/len(test_cases):6.2f} %')


 Avg. Error over 100 test cases:  48.00 %


<module 'milling_tool_environment' from 'D:\\Rajesh\\ResearchLab\\LG_\\milling_tool_environment.py'>